<a href="https://colab.research.google.com/github/dunkelweizen/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Cai_Nowicki_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [77]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df['created'] = pd.to_datetime(df['created'])

In [79]:
df['created'].loc[1]

Timestamp('2016-06-12 12:19:27')

In [0]:
train = df[df['created'] < '2016-06-01 00:00:00']

In [0]:
test = df[df['created'] >= '2016-06-01 00:00:00']

In [82]:
train.shape

(31844, 34)

In [83]:
test.shape

(16973, 34)

In [84]:
train.describe()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
count,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000,31844.000000
mean,1.203728,1.528357,40.750743,-73.972867,3575.604007,0.530430,0.477139,0.480907,0.445861,0.430725,0.418666,0.369834,0.057311,0.267586,0.185938,0.175700,0.133777,0.143983,0.104290,0.088620,0.060734,0.055929,0.051470,0.047733,0.042269,0.044216,0.039222,0.028388,0.029048
std,0.472447,1.105061,0.038658,0.028910,1762.136694,0.499081,0.499485,0.499643,0.497068,0.495185,0.493348,0.482767,0.232439,0.442707,0.389062,0.380571,0.340418,0.351078,0.305641,0.284198,0.238845,0.229788,0.220957,0.213203,0.201204,0.205577,0.194127,0.166082,0.167943
min,0.000000,0.000000,40.575700,-74.087300,1375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728500,-73.991800,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,40.751700,-73.978100,3150.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,40.773600,-73.955000,4095.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,7.000000,40.910200,-73.700100,15500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [98]:
train['pets_allowed'] = 0
for i in range(train['dogs_allowed'].count()):
  if train['dogs_allowed'].iloc[i] + train['cats_allowed'].iloc[i] == 2:
    train['pets_allowed'].iloc[i] = 1
  else:
    train['pets_allowed'].iloc[i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


KeyError: ignored

In [86]:
test['pets_allowed'] = 0
for i in range(test['dogs_allowed'].count()):
  if test['dogs_allowed'].iloc[i] + test['cats_allowed'].iloc[i] == 2:
    test['pets_allowed'].iloc[i] = 1
  else:
    test['pets_allowed'].iloc[i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [87]:
train['pets_allowed'].describe()

count    31844.000000
mean         0.445139
std          0.496989
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: pets_allowed, dtype: float64

In [88]:
train['laundry_available'] = 0
for i in range(train['laundry_in_unit'].count()):
  if train['laundry_in_unit'].iloc[i] + train['laundry_in_building'].iloc[i] >= 1:
    train['laundry_available'].iloc[i] = 1
  else:
    train['laundry_available'].iloc[i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [89]:
test['laundry_available'] = 0
for i in range(test['laundry_in_unit'].count()):
  if test['laundry_in_unit'].iloc[i] + test['laundry_in_building'].iloc[i] >= 1:
    test['laundry_available'].iloc[i] = 1
  else:
    test['laundry_available'].iloc[i] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [90]:
train.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pets_allowed,laundry_available
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
from sklearn.metrics import mean_absolute_error

In [92]:
train['price'].mean()

3575.604007034292

###Baseline Model Error

In [93]:
target = 'price'
y_train = train[target]
y_test = test[target]
guess = y_train.mean()

y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print('train error against baseline model = $', mae)

y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print('test error against baseline model = $', mae)

train error against baseline model = $ 1201.8811133682555
test error against baseline model = $ 1197.7088871089013


###Linear Regression

In [100]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

features = ['bedrooms', 'bathrooms', 'laundry_available']
target = 'price'
y_train = train[target]
y_test = test[target]
X_train = train[features]
X_test = test[features]
print(f'Linear Regression dependent on: {features}')

model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae_train = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae}')


y_pred = model.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae}')

Linear Regression dependent on: ['bedrooms', 'bathrooms', 'laundry_available']
Train Error: $819.6479260777663
Test Error: $819.6479260777663


In [95]:
model.coef_, model.intercept_

(array([ 379.36265851, 2012.50308113,  450.28568405]), 469.2381161157464)

In [0]:
train_r2 = model.score(X_train, y_train)
test_r2 = model.score(X_test, y_test)

In [97]:
print(f'R^2 for training data is {train_r2}')
print(f'R^2 for test data is {test_r2}')

R^2 for training data is 0.5223527479976087
R^2 for test data is 0.5286441595453539


In [0]:
import math
RMSE_train = math.sqrt(mae_train)
RMSE_test = math.sqrt(mae_test)

In [102]:
print('RMSE for training data is', RMSE_train)
print('RMSE for test data is', RMSE_test)

RMSE for training data is 28.472911790691136
RMSE for test data is 28.629493989202224


###More Features!

In [103]:
train['interest_level'].describe()

count     31844
unique        3
top         low
freq      22053
Name: interest_level, dtype: object

In [106]:
train['interest_level'] = train['interest_level'].replace(to_replace=('low', 'medium', 'high'), value=(1,2,3))
test['interest_level'] = test['interest_level'].replace(to_replace=('low', 'medium', 'high'), value=(1,2,3))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
def regression(train, test, features, target):
  from sklearn.linear_model import LinearRegression
  model = LinearRegression()
  y_train = train[target]
  y_test = test[target]
  X_train = train[features]
  X_test = test[features]
  print(f'Linear Regression dependent on: {features}, predicting {target}')

  model.fit(X_train, y_train)
  y_pred = model.predict(X_train)
  mae_train = mean_absolute_error(y_train, y_pred)
  print(f'Train Error: {mae_train}')
  print('R^2 value on training data:', model.score(X_train, y_train))


  y_pred = model.predict(X_test)
  mae_test = mean_absolute_error(y_test, y_pred)
  print(f'Test Error: {mae_test}')
  print('R^2 value on test data:', model.score(X_test, y_test))

In [116]:
regression(train, test, ['bedrooms', 'bathrooms', 'interest_level'], 'price')

Linear Regression dependent on: ['bedrooms', 'bathrooms', 'interest_level'], predicting price
Train Error: 791.8267165930765
R^2 value on training data: 0.5403277932326123
Test Error: 792.6617034799692
R^2 value on test data: 0.5537862677958163


In [117]:
regression(train, test, ['bedrooms', 'bathrooms'], 'price')

Linear Regression dependent on: ['bedrooms', 'bathrooms'], predicting price
Train Error: 818.5310213271714
R^2 value on training data: 0.5111543084316605
Test Error: 825.8987822403527
R^2 value on test data: 0.5213303957090346


In [118]:
regression(train, test, ['bedrooms', 'bathrooms', 'elevator', 'pets_allowed'], 'price')

Linear Regression dependent on: ['bedrooms', 'bathrooms', 'elevator', 'pets_allowed'], predicting price
Train Error: 791.0547045751715
R^2 value on training data: 0.5314808324706244
Test Error: 798.8568459132107
R^2 value on test data: 0.5401880232409575


In [119]:
regression(train, test, ['bedrooms', 'bathrooms', 'elevator', 'pets_allowed', 'laundry_available'], 'price')

Linear Regression dependent on: ['bedrooms', 'bathrooms', 'elevator', 'pets_allowed', 'laundry_available'], predicting price
Train Error: 786.1682546891418
R^2 value on training data: 0.5381167142071307
Test Error: 794.7548542908727
R^2 value on test data: 0.5438645565265686
